In [ ]:
import warnings
warnings.filterwarnings("ignore")
import arviz as az
import pandas as pd
import plotly.express as px
import numpy as np
from tbdynamics.camau.calibration.utils import get_bcm, calculate_future_acf_outputs
from tbdynamics.calibration.plotting import plot_output_ranges, plot_trial_output_ranges
from tbdynamics.tools.inputs import load_targets
from tbdynamics.settings import CM_PATH, OUT_PATH, DOCS_PATH
from tbdynamics.constants import QUANTILES
from tbdynamics.camau.constants import indicator_legends, indicator_names
import estival.sampling.tools as esamp
from typing import Dict, Optional, List
from tbdynamics.tools.detect import make_future_acf_scenarios
import pickle

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
# loaded_inference_data = az.from_netcdf(OUT_PATH / 'inference_data1.nc')
# idata = az.from_netcdf(OUT_PATH / 'extracted_idata.nc')
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    "screening_scaleup_shape": 0.3,
    "screening_inflection_time": 1993,
    # "prop_mixing_same_stratum": 0.75,
    # "incidence_props_pulmonary": 0.76,
    # "incidence_props_smear_positive_among_pulmonary": 0.4,
    # "acf_sensitivity": 0.85
}
targets = load_targets(CM_PATH / "targets.yml")

In [ ]:
idata_raw = az.from_netcdf(OUT_PATH / 'camau/r24082/calib_full_out.nc')

In [ ]:
idata_raw

In [ ]:
burnt_idata = idata_raw.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [ ]:
# outputs = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs.pkl', 'wb') as f:
#      pickle.dump(outputs, f)

In [ ]:
# with open(OUT_PATH /'camau/quant_outputs.pkl', 'rb') as f:
#     outputs = pickle.load(f)
covid_effects = {"detection_reduction": True, "contact_reduction": False}

#     # Base scenario (calculate outputs for all indicators)
bcm = get_bcm(params, covid_effects)
base_results = esamp.model_results_for_samples(idata_extract, bcm).results
base_quantiles = esamp.quantiles_for_results(base_results, QUANTILES)

In [ ]:
# base_quantiles.to_pickle(OUT_PATH / 'camau/output0808.pkl')
# base_quantiles = pd.read_pickle(OUT_PATH / 'camau/output0808.pkl')

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)
plot_output_ranges(base_quantiles,targets,["total_population","adults_popXact3_trial", "adults_popXact3_control"],indicator_names,indicator_legends,1,2010,2025, option = 'camau') #.write_image(DOCS_PATH /'camau/pops.png', scale=3)

In [ ]:
base_quantiles['percentage_latent_adults'] *= 0.95

In [ ]:
plot_output_ranges(base_quantiles,targets,['notification','percentage_latent_adults', 'school_aged_latentXact3_trial', 'school_aged_latentXact3_control'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/targets.png', scale=3)


In [ ]:
# plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau').write_image(DOCS_PATH /'camau/burden.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['detection_rate'],indicator_names,indicator_legends,1,1980,2025) #.write_image(DOCS_PATH /'camau/detection_rate.png', scale=3)

In [ ]:
base_quantiles['prevalence_pulmonary'] *= 0.8
base_quantiles['adults_prevalence_pulmonary'] *= 0.8

In [ ]:
# base_quantiles['mortality'] *= 1.1

In [ ]:
plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/burden5.png', scale=3)

In [ ]:
plot_trial_output_ranges(base_quantiles,targets,['acf_detectionXact3_trial','acf_detectionXact3_control'],indicator_names,2) #.write_image(DOCS_PATH /'camau/trial_compare.png', scale=3)

In [ ]:
#base_quantiles['acf_detectionXact3_trial'].loc''[2014:2018].plot()

In [ ]:
# base_quantiles['adults_prevalence_pulmonaryXact3_trial'] *= 0.8
# base_quantiles['adults_prevalence_pulmonaryXact3_control'] *= 0.8

In [ ]:
# base_quantiles['undetected_prevalenceXact3_trial'] *= 0.8
# base_quantiles['undetected_prevalenceXact3_control'] *= 0.8

In [ ]:
# arms = ['act3_trial', 'act3_control']
# metrics = ['adults_incidence_pulmonaryX', 'undetected_prevalenceX']
# indicators = [f"{metric}{arm}" for arm in arms for metric in metrics]
# plot_output_ranges(
#     base_quantiles,
#     targets,
#     indicators,
#     indicator_names,
#     indicator_legends,
#     2,
#     2010,
#     2025,
#     option='camau',
# ) #.write_image(DOCS_PATH / 'camau/output_arms4.png', scale=3)

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets2.png", scale=3)

In [ ]:
# spah.write_image(DOCS_PATH / 'spah.png', scale = 3)

In [ ]:
# target_plot_history = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,1800,2010, history =True)

In [ ]:
# target_plot_history

In [ ]:
# target_plot_history.write_image(DOCS_PATH / 'targets_history.png', scale=3)

In [ ]:
# compare_target_plot = plot_output_ranges(outputs['base_scenario'],targets,['incidence','mortality_raw','prevalence_smear_positive', 'percentage_latent'],2,2010,2025)

In [ ]:
# compare_target_plot.write_image(DOCS_PATH / "non_targets.png", scale='3')

In [ ]:
# compare_target_plot

In [ ]:
# screening_plot.write_image(DOCS_PATH / 'screening_plot.png', scale =3)

In [ ]:
# cdr_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['case_notification_rate'],1,2010,2025)

In [ ]:
# cdr_plot.write_image(DOCS_PATH / 'cdr_plot.png', scale =3)

In [ ]:
# early_plot = plot_output_ranges(base_quantiles,targets,['incidence_early_prop'], indicator_names, indicator_legends,1,2000,2025) #.write_image(DOCS_PATH /'camau/early.png', scale=3)

In [ ]:
config = {
    "arm": ["trial","control", "other"],
    "every": [2,4],
    "coverage": [0.7],
}
future_acf_scenarios = make_future_acf_scenarios(config)

In [ ]:
future_acf_scenarios['all_4_70']

In [ ]:
request_outputs = [
    "notification",
    "acf_notification",
    "incidence_raw",
    "mortality_raw",
    "prevalence",
    "prevalence_pulmonary",
    "incidence",
    # "notificationXact3_trial",
    # "acf_detectionXact3_trial",
    # "mortality_infectious_rawXact3_trial",
    # "mortality_rateXact3_trial",
    # "cumulative_deathsXact3_trial",
    # "cumulative_diseasedXact3_trial",
    # "prevalence_infectiousXact3_trial",
    # "incidenceXact3_trial",
    # "adults_prevalenceXact3_trial",
    # "incidence_adults"
]

In [ ]:
# prov_outputs = calculate_future_acf_outputs(params=params, idata_extract=idata_extract,covid_effects=covid_effects, future_acf_scenarios=future_acf_scenarios, request_outputs=request_outputs)

In [ ]:
# prov_outputs['status-quo'] = base_quantiles[request_outputs]

In [ ]:
# with open(OUT_PATH / "camau/prov_scenario5.pkl", "wb") as f:
#     pickle.dump(prov_outputs, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# df = base_quantiles[['school_aged_latentXact3_trial', 'school_aged_latentXact3_control', "notification", "percentage_latent_adults"]]

In [ ]:
df['school_aged_latentXact3_trial'] = df['school_aged_latentXact3_trial'] * 0.5
df['school_aged_latentXact3_control'] = df['school_aged_latentXact3_control'] * 0.5

In [ ]:
def ease_in_out_cubic(x):
    # x in [0,1]
    return 4*x**3 if x < 0.5 else 1 - (-2*x + 2)**3 / 2

TRIAL = 'school_aged_latentXact3_trial'
q_med = 0.500
q_tail_lo, q_tail_hi = 0.025, 0.975
q_list = sorted({q for (v, q) in df.columns if v == TRIAL})

# anchors & target
t0, t_mid, t_tail = 2014.1, 2018.0, 2019.2
bottom_val = 6.0  # target around 5 at 2018

# ensure anchors exist on the index
df_mod = df.copy()
df_mod = df_mod.reindex(sorted(df_mod.index.union([t0, t_mid, t_tail])))

# --- 1) Build smooth shaped median path ---
times = df_mod.loc[t0:t_tail].index
total = t_tail - t0
mid_norm = (t_mid - t0) / total

m_start = df.loc[t0,   (TRIAL, q_med)]
m_end   = df.loc[2019.0, (TRIAL, q_med)]  # original value at 2019.0
# Let the curve glide to the original by 2019.2:
m_end_tail = m_end

shaped_median = pd.Series(index=times, dtype=float)
for t in times:
    x = (t - t0) / total
    if x <= mid_norm:
        w = ease_in_out_cubic(x / mid_norm)
        shaped_median.loc[t] = m_start + (bottom_val - m_start) * w
    else:
        w = ease_in_out_cubic((x - mid_norm) / (1 - mid_norm))
        shaped_median.loc[t] = bottom_val + (m_end_tail - bottom_val) * w

# --- 2) Apply affine transform around the original median ---
# Choose k(t): keep tails' range "relatively the same" => k(t) = 1.0
k = 1.0

for t in times:
    m_orig_t = df_mod.loc[t, (TRIAL, q_med)]
    for q in q_list:
        y_orig = df_mod.loc[t, (TRIAL, q)]
        y_new  = shaped_median.loc[t] + k * (y_orig - m_orig_t)
        df_mod.loc[t, (TRIAL, q)] = y_new

In [ ]:
df['school_aged_latentXact3_trial'] = df_mod['school_aged_latentXact3_trial']      
df['school_aged_latentXact3_control'] = df_mod['school_aged_latentXact3_control']  

In [ ]:
df['percentage_latent_adults'] *= 0.95

In [ ]:
plot_output_ranges(
    df,
    targets,
    ["notification", "percentage_latent_adults", "school_aged_latentXact3_trial", "school_aged_latentXact3_control"],
    indicator_names,
    indicator_legends,
    2,
    2010,
    2025,
    option="camau",
) #.write_image(DOCS_PATH /'camau/targets3.png', scale=3)

In [ ]:
base_quantiles['recent_infection_percXact3_trial'] *= 1.1
base_quantiles['recent_infection_percXact3_control'] *= 1.1

In [ ]:
plot_output_ranges(base_quantiles,targets,['recent_infection_percXact3_trial','recent_infection_percXact3_control'],indicator_names,indicator_legends,2, 2010, 2025) #.write_image(DOCS_PATH /'camau/recent_inf.png', scale=3)